In [1]:
import pandas as pd
import numpy as np
import cornac.models.recommender as cmr
import math

In [2]:
def mag(x): return math.sqrt(sum(i**2 for i in x))

In [3]:
def cosine_similarity(p_m, u_m, user_id):
    cos_sim = []
    
    u_vec = u_m.loc[user_id,:]
    u_size = mag(u_vec)

    for i in range(len(p_m)):
        p_vec = p_m.iloc[i,:]
        p_size = mag(p_vec)
        cos_sim.append(np.dot(p_vec, u_vec)/(p_size*u_size))
        
    return cos_sim

In [14]:
#Eat_contents_based
def Eat_recommendation(u_id):
     # 0. preprocess place_matrix
    place_matrix = pd.read_csv('../../Eat/data/normalized_item_matrix.csv')
    cbf = place_matrix[['p_id']]
    place_matrix.set_index('p_id', inplace=True)
    
     # 1. preprocess user_matrix
    user_matrix = pd.read_csv('../../Eat/data/normalized_user_matrix.csv')
    user_id  = 'u1s2&'+u_id
    user_index = user_matrix[user_matrix['u_id']==user_id].index[0]
    del user_matrix['u_id']
    
    # 2. cbf result
    cbf_result = cosine_similarity(place_matrix, user_matrix, user_index)
    cbf['cbf'] = cbf_result
    cbf.set_index('p_id', inplace=True, drop=True)
    print('Complete to get Eat - contents_based result')
    
    # 3. ncf result
    ncf = pd.read_csv('../../Eat/data/predictions/ncf_scores.csv')
    ncf.rename(columns = {'ncf_score':'ncf'}, inplace=True)
    ncf = ncf[ncf['u_id'] == u_id]
    del ncf['u_id']
    ncf['p_id'] = ncf['p_id'].astype(int)
    ncf.set_index('p_id', inplace=True, drop=True)
    print('Complete to get Eat - ncf result')
    
    # 4. make_matrix
    eat_info =  pd.read_csv('../../Eat/data/translated_eat_info.csv')
    eat_info = pd.DataFrame(eat_info, columns=['p_id','name','address'])
    eat_info.set_index('p_id', inplace=True, drop=True)
    eat_info['district'] = 0
    for i in range (0, len(eat_info['address'])):
        eat_info['district'][i] = eat_info['address'][i].split(' ')[1]
    print('Complete to make Eat -Result matrix')
    
    # 5. join more
    result = eat_info.join(cbf, how='left').join(ncf, how='left')
    result['result'] = result['cbf'] * result['ncf']
#     result.sort_values(by=['p_id'], inplace=True, ascending=True)
    del result['district']
    del result['address']
    
    return result.loc[0:25];

In [15]:
Eat_recommendation('쥬')

/Users/ryeonju/miniconda3/envs/reco_base/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3254: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
/Users/ryeonju/miniconda3/envs/reco_base/lib/python3.6/site-packages/ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in double_scalars
  # Remove the CWD from sys.path while we load stuff.
/Users/ryeonju/miniconda3/envs/reco_base/lib/python3.6/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


Complete to get Eat - contents_based result
Complete to get Eat - ncf result


/Users/ryeonju/miniconda3/envs/reco_base/lib/python3.6/site-packages/ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/ryeonju/miniconda3/envs/reco_base/lib/python3.6/site-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


Complete to make Eat -Result matrix


,name,cbf,ncf,result
p_id,,,,
0,(휴업중) 유라꾸키친,0.230494,1.400709e-06,3.228556e-07
1559,맨프롬오키나와,0.068038,1.123548e-05,7.644356e-07
180,감각,0.204923,1.546741e-05,3.169630e-06
1120,두남자국물닭갈비,0.083465,1.490116e-07,1.243732e-08
3360,오가와,0.361213,3.337860e-06,1.205679e-06
...,...,...,...,...
4405,침사추이누들,0.156316,3.278255e-07,5.124432e-08
3807,이야초밥,NaN,9.179115e-06,NaN
1263,란주칼면,0.092518,2.406538e-04,2.226481e-05


In [48]:
#Go_contents_based
def Go_recommendation(u_id):
    # 0. preprocess place_matrix
    place_matrix = pd.read_csv('../../Go/data/normalized_item_matrix.csv')
    cbf = place_matrix[['p_id']]
    del place_matrix['total_count']
    place_matrix.set_index('p_id', inplace=True)
    
    # 1. preprocess user_matrix
    user_matrix = pd.read_csv('../../Go/data/normalized_user_matrix.csv')
    user_id  = 'u1s2&'+u_id
    user_index = user_matrix[user_matrix['u_id']==user_id].index[0]
    del user_matrix['u_id']
    del user_matrix['total_count']
    
    # 2. cbf result
    cbf_result = cosine_similarity(place_matrix, user_matrix, user_index)
    cbf['cbf'] = cbf_result
    cbf.set_index('p_id', inplace=True, drop=True)
    print('Complete to get Go - contents_based result')
    
    # 3. ncf result
    ncf = pd.read_csv('../../Go/data/ncf_scores2.csv')
    ncf.rename(columns = {'ncf_score':'ncf'}, inplace=True)
    ncf = ncf[ncf['u_id'] == u_id]
    del ncf['u_id']
    ncf.set_index('p_id', inplace=True, drop=True)
    print('Complete to get Go - ncf result')
    
    # 4. make_matrix
    go_info =  pd.read_csv('../../Go/data/item_info.csv')
    go_info = pd.DataFrame(go_info, columns=['p_id','p_name','address'])
    go_info['district'] = 0;
    for i in range (0, len(go_info['address'])):
        try:
            if ' ' in go_info['address'][i]:
                go_info['district'][i] = go_info['address'][i].split(' ')[1]
            else:
                go_info['district'][i] = go_info['address'][i]
        except:
            pass
    go_info.set_index('p_id', inplace=True, drop=True)
    print('Complete to make Go - Result matrix')
    
    # 5. join more
    result = go_info.join(cbf, how='left').join(ncf, how='left')
    result['result'] = result['cbf'] * result['ncf']
    result.dropna(inplace=True)
    result.sort_values(by=['ncf'], inplace=True, ascending=False)
    del result['district']
    del result['address']
    del result['p_name']
    return result;

In [49]:
Go_recommendation('쥬')

/Users/ryeonju/miniconda3/envs/reco_base/lib/python3.6/site-packages/ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in double_scalars
  # Remove the CWD from sys.path while we load stuff.
/Users/ryeonju/miniconda3/envs/reco_base/lib/python3.6/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Complete to get Go - contents_based result
Complete to get Go - ncf result


/Users/ryeonju/miniconda3/envs/reco_base/lib/python3.6/site-packages/ipykernel_launcher.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/ryeonju/miniconda3/envs/reco_base/lib/python3.6/site-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


Complete to make Go - Result matrix


,cbf,ncf,result
p_id,,,
g239,0.000000,9.937845e-01,0.000000e+00
g84,0.171600,2.636588e-01,4.524373e-02
f231,0.244371,7.537377e-02,1.841918e-02
3,0.222064,6.744722e-02,1.497761e-02
e32,0.089166,3.127259e-02,2.788444e-03
...,...,...,...
a314,0.106336,6.556511e-07,6.971955e-08
152,0.000000,5.066395e-07,0.000000e+00
d114,0.343199,4.768372e-07,1.636501e-07


In [47]:
#Watch_contents_based
def Watch_recommendation(u_id):
    # 0. preprocess place_matrix
    place_matrix = pd.read_csv('../../Watch/data/normalized_item_matrix.csv')
    cbf = place_matrix[['p_id']]
    place_matrix.set_index('p_id', inplace=True)
    
     # 1. preprocess user_matrix
    user_matrix = pd.read_csv('../../Watch/data/normalized_user_matrix.csv')
    user_id  = 'u1s2&'+u_id
    user_index = user_matrix[user_matrix['u_id']==user_id].index[0]
    del user_matrix['u_id']
    
    # 2. cbf result
    cbf_result = cosine_similarity(place_matrix, user_matrix, user_index)
    cbf['cbf'] = cbf_result
    cbf.set_index('p_id', inplace=True, drop=True)
    print('Complete to get Watch - contents_based result')
    
    # 3. ncf result
    ncf = pd.read_csv('../../Watch/data/predictions/ncf_scores.csv')
    ncf.rename(columns = {'ncf_score':'ncf'}, inplace=True)
    ncf = ncf[ncf['u_id'] == u_id]
    del ncf['u_id']
    ncf['p_id'] = ncf['p_id'].astype(int)
    ncf.set_index('p_id', inplace=True, drop=True)
    print('Complete to get Watch - ncf result')
    
    # 4. make_matrix
    watch_info =  pd.read_csv('../../Watch/data/current_movie.csv')
    watch_info = pd.DataFrame(watch_info, columns=['id','name'])
    watch_info.set_index('id', inplace=True, drop=True)
    print('Complete to make Watch Result matrix')
    
    # 5. join cbf,ncf
    result = watch_info.join(cbf, how='left').join(ncf, how='left')
    result['result'] = result['cbf'] * result['ncf']
    
    # 6.get movie_name
    result.sort_values(by=['result'], ascending=False, inplace=True)
    reco_movie_name = result.iloc[0]['name']
    
    return reco_movie_name;

In [9]:
def get_preferred_district(u_id):
    user_survey_df = pd.read_csv('../data/survey_result.csv')
    district = user_survey_df[user_survey_df['u_id'] == u_id]['District'].values[0]
    return district.split(', ')

In [10]:
def Search_Recommendable_Places(eat_matrix, go_matrix, u_id):
    dist_ls = get_preferred_district(u_id)
    eat_matrix = eat_matrix[eat_matrix['district'].isin(dist_ls)]
    go_matrix = go_matrix[go_matrix['district'].isin(dist_ls)]
    
    eat_matrix.reset_index(inplace=True, drop=True)
    go_matrix.reset_index(inplace=True, drop=True)
    
    trial=0
    e_i=0
    g_i=0
    
    if len(dist_ls) > 1 :
        if (eat_matrix['district'][0] == go_matrix['district'][0]):
            print('베스트 행정구 일치!')
            eat_res =  eat_matrix.iloc[0]
            go_res = go_matrix.iloc[0]
            return (eat_res, go_res)
        
        else:
            while (eat_matrix['district'][0] != go_matrix['district'][g_i]):
                g_i = g_i + 1
            eat_res1 =  eat_matrix.iloc[0]
            go_res1 = go_matrix.iloc[g_i]
            
            while (eat_matrix['district'][e_i] != go_matrix['district'][0]):
                e_i = e_i + 1
            eat_res2 =  eat_matrix.iloc[e_i]
            go_res2 = go_matrix.iloc[0]
            
            return(eat_res1, go_res1, eat_res2, go_res2)
        
#         while (e_i< len(eat_matrix) and g_i<len(go_matrix)):
#             if (eat_matrix['district'][e_i] == go_matrix['district'][g_i]):
#                 eat_res =  eat_matrix.iloc[e_i]
#                 go_res = go_matrix.iloc[g_i]
#                 return (eat_res, go_res)
                
#             if (trial%2 == 0):
#                 e_i = e_i+1
#             else:
#                 g_i = g_i+1
#             trial = trial + 1
    else:
        eat_res = eat_matrix[eat_matrix['district']==dist_ls[0]].iloc[0]
        go_res = go_matrix[go_matrix['district']==dist_ls[0]].iloc[0]
        return (eat_res, go_res)

In [11]:
def save_each_user_reco_item(user_id, eat, go, watch):
    pd.to_csv(f'../data/')

In [12]:
def main():
    u_id = input('Enter User ID: ')
    go = Go_recommendation(u_id)
    eat = Eat_recommendation(u_id)
    watch = Watch_recommendation(u_id)
    result = Search_Recommendable_Places(eat, go, u_id)
    if len(result) == 2:
        eat_result = result[0]
        go_result = result[1]
        print(f'{u_id}님께 추천드릴 맛집은 {eat_result["district"]}에 위치한 {eat_result["name"]}입니다.')
        print(f'{u_id}님께 추천드릴 데이트 장소는 {go_result["district"]}에 위치한 {go_result["p_name"]}입니다.')

    elif len(result) == 4:
        eat_result1 = result[0]
        go_result1 = result[1]
        eat_result2 = result[2]
        go_result2 = result[3]
        print(f'{u_id}님! {eat_result1["district"]}에서의 데이트를 계획하고 계신가요?')
        print(f'{u_id}님께 추천드릴 맛집은 {eat_result1["district"]}에 위치한 {eat_result1["name"]}입니다.')
        print(f'{u_id}님께 추천드릴 데이트 장소는 {go_result1["district"]}에 위치한 {go_result1["p_name"]}입니다.')
        print(f'혹시나! {eat_result2["district"]}에서의 데이트를 계획하고 계신다면?')
        print(f'맛집으로는 {eat_result2["district"]}에 위치한 {eat_result2["name"]}와 ')
        print(f'데이트 장소로는 {go_result2["district"]}에 위치한 {go_result2["p_name"]}을(를) 추천드려요.')
        
    print(f'영화가 끌리는 데이트라면 현재 상영중인 영화 {watch}은(는) 어떠신가요?')

In [ ]:
main()